In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the dependencies

In [15]:
import os
import random
import shutil
import numpy as np
import cv2  # OpenCV for image processing
import torch  # Assuming YOLOv5 is being used
import tensorflow as tf
import pandas as pd

# Set paths

In [3]:
train_images_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Images/train/'
bounding_boxes_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Bounding_boxes_YOLO/train/'
output_images_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Images/selected_train/'
output_labels_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Bounding_boxes_YOLO/selected_train/'

# Create output directories
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Load Images

In [5]:
# Get all image paths
image_files = [f for f in os.listdir(train_images_dir) if f.endswith('.jpg')]

# Randomly select 400 images
selected_images = random.sample(image_files, 400)

# Ensure output directories exist
os.makedirs(output_images_dir, exist_ok=True)

# Copy selected images and their corresponding labels to the new directory
for image_file in selected_images:
    # Copy image
    shutil.copy(os.path.join(train_images_dir, image_file), output_images_dir)

    # Create corresponding label file name
    label_file = image_file.replace('.jpg', '.txt')

    # Check if the corresponding label file exists and copy it
    label_file_path = os.path.join(train_images_dir, label_file)
    if os.path.exists(label_file_path):
        shutil.copy(label_file_path, output_images_dir)
    else:
        print(f"Label file {label_file} does not exist, skipping.")

print("Selected images and corresponding labels have been copied successfully.")

Selected images and corresponding labels have been copied successfully.


# 2. Train YOLO Model

In [6]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17018, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 17018 (delta 120), reused 124 (delta 68), pack-reused 16805 (from 1)
Receiving objects: 100% (17018/17018), 15.73 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (11658/11658), done.


### Change Directory to YOLOv5

In [7]:
os.chdir('yolov5')

In [22]:
!pwd

/content/yolov5


### Train the model

In [8]:
# Define dataset YAML
yaml_content = """
train: /content/drive/My Drive/COS40007/Week6Portfolio/Images/selected_train/
val: /content/drive/My Drive/COS40007/Week6Portfolio/Images/train/
nc: 1
names: ['Graffiti']
"""

# Write the YAML file
with open('graffiti.yaml', 'w') as f:
    f.write(yaml_content)

print("Dataset YAML file created.")

Dataset YAML file created.


In [9]:
!python train.py --img 640 --batch 16 --epochs 50 --data graffiti.yaml --weights yolov5s.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 48.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-10-20 08:48:59.285761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 08:48:59.305112: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 08:48:59.311494

# 3. Randomly take 40 images from test data and compute IoU for each and generate a CSV file containing 3 columns

In [14]:
# Load your YOLO model (adjust the path to your model)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt')  # Load custom model
model.eval()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-20 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

### Select random 40 images

In [19]:
# Set paths
test_images_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Images/test'
bounding_boxes_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Images/test/'
output_test_images_dir = '/content/drive/My Drive/COS40007/Week6Portfolio/Images/selected_test/'
os.makedirs(output_test_images_dir, exist_ok=True)
# Get all test image paths
test_image_files = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg')]

# Randomly select 40 images
selected_test_images = random.sample(test_image_files, 40)

# Copy selected images and their corresponding label files to the output directory
for image_file in selected_test_images:
    # Copy the image file
    shutil.copy(os.path.join(test_images_dir, image_file), os.path.join(output_test_images_dir, image_file))

    # Determine the corresponding label file
    label_file = image_file.replace('.jpg', '.txt')
    label_file_path = os.path.join(bounding_boxes_dir, label_file)

    # Check if the label file exists
    if os.path.exists(label_file_path):
        shutil.copy(label_file_path, os.path.join(output_test_images_dir, label_file))
        print(f"Copied label file: {label_file}")
    else:
        print(f"Label file not found for: {image_file}")

print(f"Copied {len(selected_test_images)} images and their labels to {output_test_images_dir}.")

Copied label file: IMG_20180719_143637.txt
Copied label file: IMG_20180718_105313.txt
Copied label file: IMG_20180714_103316.txt
Copied label file: IMG_20180724_125116.txt
Copied label file: IMG_20180721_090501.txt
Copied label file: IMG_20100101_033113.txt
Copied label file: IMG_20180810_085645.txt
Copied label file: IMG_20180721_090151.txt
Copied label file: IMG_20180724_123854.txt
Copied label file: IMG_20180724_150928.txt
Copied label file: IMG_20180719_123848.txt
Copied label file: IMG_20180724_120949.txt
Copied label file: IMG_20180723_122303.txt
Copied label file: IMG_20180724_130100.txt
Copied label file: IMG_20180724_122617.txt
Copied label file: IMG_20180717_122754.txt
Copied label file: IMG_20100101_022751.txt
Copied label file: IMG_20180724_150833.txt
Copied label file: IMG_20180720_142545.txt
Copied label file: IMG_20180723_123225.txt
Copied label file: IMG_20180714_102729.txt
Copied label file: IMG_20180720_142150.txt
Copied label file: IMG_20180723_145744.txt
Copied labe

### Load 40 Images

In [20]:
fourty_selected_images = [f for f in os.listdir(test_images_dir) if f.endswith('.jpg')]

In [24]:
import os
import numpy as np
import cv2
import pandas as pd

def compute_iou(true_boxes, pred_boxes):
    # Convert to numpy arrays for easier calculations
    true_boxes = np.array(true_boxes)
    pred_boxes = np.array(pred_boxes)

    # Check for empty boxes
    if true_boxes.shape[0] == 0 or pred_boxes.shape[0] == 0:
        return np.zeros(0)  # No IoU if there are no boxes

    # Calculate IoU
    x1 = np.maximum(true_boxes[:, 0][:, None], pred_boxes[:, 0])  # Shape (N, M)
    y1 = np.maximum(true_boxes[:, 1][:, None], pred_boxes[:, 1])
    x2 = np.minimum(true_boxes[:, 2][:, None], pred_boxes[:, 2])
    y2 = np.minimum(true_boxes[:, 3][:, None], pred_boxes[:, 3])

    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    true_area = (true_boxes[:, 2] - true_boxes[:, 0]) * (true_boxes[:, 3] - true_boxes[:, 1])
    pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])

    union_area = true_area[:, None] + pred_area - intersection_area  # Shape (N, M)
    iou = intersection_area / union_area
    return iou

final_results = []

for image_file in fourty_selected_images:
    # Load the image
    img_path = os.path.join(test_images_dir, image_file)
    img = cv2.imread(img_path)

    # Perform inference
    results = model(img)

    # Get the predictions
    preds = results.pred[0]  # Predictions for the first image
    pred_boxes = []

    for *box, conf in preds.tolist():
        pred_boxes.append(box)

    # Load corresponding ground truth box
    label_file = image_file.replace('.jpg', '.txt')
    true_boxes = []

    if os.path.exists(os.path.join(bounding_boxes_dir, label_file)):
        with open(os.path.join(bounding_boxes_dir, label_file), 'r') as f:
            for line in f.readlines():
                parts = list(map(float, line.strip().split()))
                # Convert YOLO format (class_id, x_center, y_center, width, height) to (xmin, ymin, xmax, ymax)
                class_id, x_center, y_center, width, height = parts
                xmin = (x_center - width / 2) * img.shape[1]
                ymin = (y_center - height / 2) * img.shape[0]
                xmax = (x_center + width / 2) * img.shape[1]
                ymax = (y_center + height / 2) * img.shape[0]
                true_boxes.append([xmin, ymin, xmax, ymax])

    results_data = {
        'image_name': image_file,
        'confidence_value': 0,  # Default to 0
        'IoU_value': 0  # Default to 0
    }

    # Calculate IoU if there are predictions
    if pred_boxes:
        if true_boxes:
            iou_values = compute_iou(true_boxes, pred_boxes)
            if iou_values.size > 0:  # Ensure there are IoU values
              results_data['IoU_value'] = np.max(iou_values)  # Get the max IoU
            else:
              results_data['IoU_value'] = 0  # No valid IoU values
        else:
            results_data['IoU_value'] = 0  # No ground truth boxes
    final_results.append(results_data)

# Create a DataFrame from the results
results_df = pd.DataFrame(final_results)

# Save to CSV
results_df.to_csv('detection_results.csv', index=False)
print("Results saved to detection_results.csv")

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

Results saved to detection_results.csv


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
